In [1]:
using ITensors
using FiniteSizeScaling
using ProgressMeter
using JLD2

In [2]:
function DMRG(N; g)
    sites = siteinds("S=1/2",N)
    
    H = OpSum()
    for i in 1:N-1
        H += -1,"Sz",i,"Sz",i+1
    end
    for i in 1:N
        H += -g,"Sx",i
    end
    
    MPH = MPO(H,sites)
    ψ = randomMPS(sites, 10)
    
    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,200,400,800)
    setcutoff!(sweeps, 1E-8)
    E_0, ψ_0 = dmrg(MPH, ψ, sweeps, outputlevel = 0)
    
    return ψ_0
end

DMRG (generic function with 1 method)

In [3]:
function binder_cumulant(;MPS)
    sites = siteinds(MPS)
    L = length(sites)
    S = collect(1:L)
    
    os = OpSum()
    for j=1:L
        os += 1/L,"Sz",j
    end
    M = MPO(os,sites)
    M² = apply(M,M)
    M⁴ = apply(M²,M²)
    
    U = 1 - inner(MPS',M⁴,MPS)/(3*inner(MPS',M²,MPS)^2)
    
    return U
end

binder_cumulant (generic function with 1 method)

In [4]:
g = collect(0.0:0.01:1.0)
L = [60,65,70,75,80]
BC_data = Vector{Vector{Float64}}(undef, length(L))
φ = Vector{MPS}(undef, length(g))

for l in 1:length(L)
    BC = fill(0.0, length(g))
    @showprogress for i in 1:length(g)
        @inbounds φ[i] = DMRG(L[l], g = g[i])
        @inbounds BC[i] += binder_cumulant(MPS = φ[i])
    end
    @inbounds BC_data[l] = BC
end

Progress: 100%|█████████████████████████████████████████| Time: 0:02:36
Progress: 100%|█████████████████████████████████████████| Time: 0:01:47
Progress: 100%|█████████████████████████████████████████| Time: 0:01:57
Progress: 100%|█████████████████████████████████████████| Time: 0:02:10
Progress: 100%|█████████████████████████████████████████| Time: 0:02:20


In [5]:
BC = hcat(BC_data...)
save_object("BinderCumulants.jld2",BC)

In [6]:
function nu(BinderCumulant; data = BC, g_c = 0.5)
    BC_data = BinderCumulant
    fsi = Matrix{Int64}(undef, 2, length(sizes))
    
    for j in 1:length(sizes)
        start = 1 
        stop = 1 
        for i in 1:length(g)
            if data[i,j] > 0.60
                start = i
            end
            if data[i,j] > 0.35
                stop = i
            end
        end
        fsi[:,j]=[start; stop]
    end
    
    BC_FSS = [[g[fsi[1,i]:fsi[2,i]], data[fsi[1,i]:fsi[2,i],i], sizes[i]] for i in 1:length(sizes)]
    x_scaled(X, L, v1, v2) = ((X.-v1)./v1)*L^(1/v2)
    y_scaled(Y, L, v1, v2) = Y

    scaled_data, residuals, min_res, best_v1, best_v2 = fss_two_var(data = BC_FSS, xs = x_scaled, ys = y_scaled, 
        v1i = g_c, v1f = g_c, n1 = 1, v2i = 0.5, v2f = 1.5, n2 = 5000, p = 8)
    
    return best_v2
end

nu (generic function with 1 method)

In [7]:
v = nu(BC_data)
save_object("ν.jld2",v)

Optimal v1 value: 0.5
Optimal v2 value: 1.0521104220844169
Smallest residual: 9.156792864805343e-7 
 


In [ ]:
g_c = 0.5
L = [60:5:80;]
g = collect(0.0:0.01:1.0)
ν = load_object("ν.jld2")

In [ ]:
x_scaled(X, L, v1, v2) = ((X.-g_c)./g_c)*L^(1/ν)
y_scaled(Y, L, v1, v2) = Y